# UT Refundable EITC - Household Example Calculations

This notebook verifies the UT refundable EITC reform is working correctly by examining individual household examples.

## Reform Summary
- **Baseline**: Utah EITC = 20% of federal EITC (non-refundable, limited by tax liability)
- **Reform**: Makes the UT EITC fully refundable for households with a qualifying child age 2 or younger

## Expected Behavior
1. Households with child age <= 2: Full UT EITC becomes refundable
2. Households with children only age 3+: UT EITC remains non-refundable
3. Childless households: UT EITC remains non-refundable

In [13]:
from policyengine_us import Microsimulation, Simulation
from policyengine_core.reforms import Reform
from policyengine_us.reforms.states.ut.ut_refundable_eitc import ut_refundable_eitc
import pandas as pd
import numpy as np

In [14]:
def create_reform():
    """Reform: Make UT EITC refundable for households with children age 2 or younger"""
    param_reform = Reform.from_dict(
        {
            "gov.contrib.states.ut.eitc.in_effect": {
                "2025-01-01.2100-12-31": True
            }
        },
        country_id="us",
    )
    return (param_reform, ut_refundable_eitc)

## Example 1: Single Parent with 1-year-old (Should Benefit)

Low-income single parent with an infant - should get full refundable EITC

In [15]:
situation_1 = {
    "people": {
        "parent": {"age": {2025: 28}, "employment_income": {2025: 20_000}},
        "child": {"age": {2025: 1}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_1 = Simulation(situation=situation_1)
reform_1 = Simulation(situation=situation_1, reform=create_reform())

print("="*70)
print("EXAMPLE 1: Single Parent with 1-year-old, $20k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_1.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_1.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_1.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_1.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_1.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_1.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_1.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_1.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_1.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_1.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_1.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_1.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_1.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_1.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_1.calculate('household_net_income', 2025)[0] - baseline_1.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 1: Single Parent with 1-year-old, $20k income

Federal EITC: $4,328.00
UT EITC (20% of federal): $865.60

Baseline (non-refundable):
  UT income tax before credits: $900.00
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $48,480.62

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $865.60
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $865.60


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $-865.60
  Household net income: $49,346.22

>>> BENEFIT FROM REFORM: $865.60


## Example 2: Single Parent with 5-year-old (Should NOT Benefit)

In [16]:
situation_2 = {
    "people": {
        "parent": {"age": {2025: 30}, "employment_income": {2025: 20_000}},
        "child": {"age": {2025: 5}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_2 = Simulation(situation=situation_2)
reform_2 = Simulation(situation=situation_2, reform=create_reform())

print("="*70)
print("EXAMPLE 2: Single Parent with 5-year-old, $20k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_2.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_2.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_2.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_2.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_2.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_2.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_2.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_2.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_2.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_2.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_2.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_2.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_2.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_2.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_2.calculate('household_net_income', 2025)[0] - baseline_2.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 2: Single Parent with 5-year-old, $20k income

Federal EITC: $4,328.00
UT EITC (20% of federal): $865.60

Baseline (non-refundable):
  UT income tax before credits: $900.00
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $28,016.10

Reform (refundable for age <= 2):
  Has qualifying child: False
  UT refundable EITC: $0.00
  UT non-refundable EITC: $865.60
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $28,016.10

>>> BENEFIT FROM REFORM: $0.00


## Example 3: Married Couple with Infant (Should Benefit)

In [17]:
situation_3 = {
    "people": {
        "parent1": {"age": {2025: 25}, "employment_income": {2025: 15_000}},
        "parent2": {"age": {2025: 26}, "employment_income": {2025: 0}},
        "baby": {"age": {2025: 0}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent1", "parent2", "baby"]}},
    "families": {"family": {"members": ["parent1", "parent2", "baby"]}},
    "marital_units": {"marital_unit": {"members": ["parent1", "parent2"]}},
    "spm_units": {"spm_unit": {"members": ["parent1", "parent2", "baby"]}},
    "households": {"household": {"members": ["parent1", "parent2", "baby"], "state_code": {2025: "UT"}}},
}

baseline_3 = Simulation(situation=situation_3)
reform_3 = Simulation(situation=situation_3, reform=create_reform())

print("="*70)
print("EXAMPLE 3: Married Couple with Infant (age 0), $15k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_3.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_3.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_3.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_3.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_3.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_3.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_3.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_3.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_3.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_3.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_3.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_3.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_3.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_3.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_3.calculate('household_net_income', 2025)[0] - baseline_3.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 3: Married Couple with Infant (age 0), $15k income

Federal EITC: $4,328.00
UT EITC (20% of federal): $865.60

Baseline (non-refundable):
  UT income tax before credits: $675.00
  UT non-refundable credits: $1,965.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $49,530.96

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $865.60
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,100.00
  UT refundable credits: $865.60


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $-865.60
  Household net income: $50,396.56

>>> BENEFIT FROM REFORM: $865.60


## Example 4: Mixed Ages - One Qualifying, One Not (Should Benefit)

In [18]:
situation_4 = {
    "people": {
        "parent": {"age": {2025: 32}, "employment_income": {2025: 25_000}},
        "child1": {"age": {2025: 1}, "is_tax_unit_dependent": {2025: True}},
        "child2": {"age": {2025: 7}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child1", "child2"]}},
    "households": {"household": {"members": ["parent", "child1", "child2"], "state_code": {2025: "UT"}}},
}

baseline_4 = Simulation(situation=situation_4)
reform_4 = Simulation(situation=situation_4, reform=create_reform())

print("="*70)
print("EXAMPLE 4: Single Parent with kids age 1 and 7, $25k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_4.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_4.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_4.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_4.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_4.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_4.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_4.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_4.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_4.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_4.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_4.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_4.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_4.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_4.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_4.calculate('household_net_income', 2025)[0] - baseline_4.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 4: Single Parent with kids age 1 and 7, $25k income

Federal EITC: $6,804.51
UT EITC (20% of federal): $1,360.90

Baseline (non-refundable):
  UT income tax before credits: $1,125.00
  UT non-refundable credits: $2,360.90
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $59,971.83

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $1,360.90
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $1,360.90


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $-1,360.90
  Household net income: $61,332.73

>>> BENEFIT FROM REFORM: $1,360.91


## Example 5: Child at Age Boundary (age 2 - Should Benefit)

In [19]:
situation_5 = {
    "people": {
        "parent": {"age": {2025: 29}, "employment_income": {2025: 18_000}},
        "child": {"age": {2025: 2}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_5 = Simulation(situation=situation_5)
reform_5 = Simulation(situation=situation_5, reform=create_reform())

print("="*70)
print("EXAMPLE 5: Single Parent with child age 2, $18k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_5.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_5.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_5.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_5.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_5.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_5.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_5.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_5.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_5.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_5.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_5.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_5.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_5.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_5.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_5.calculate('household_net_income', 2025)[0] - baseline_5.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 5: Single Parent with child age 2, $18k income

Federal EITC: $4,328.00
UT EITC (20% of federal): $865.60

Baseline (non-refundable):
  UT income tax before credits: $810.00
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $47,112.42

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $865.60
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $865.60


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $-865.60
  Household net income: $47,978.02

>>> BENEFIT FROM REFORM: $865.60


## Example 6: Child Just Over Age Boundary (age 3 - Should NOT Benefit)

In [20]:
situation_6 = {
    "people": {
        "parent": {"age": {2025: 29}, "employment_income": {2025: 18_000}},
        "child": {"age": {2025: 3}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_6 = Simulation(situation=situation_6)
reform_6 = Simulation(situation=situation_6, reform=create_reform())

print("="*70)
print("EXAMPLE 6: Single Parent with child age 3, $18k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_6.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_6.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_6.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_6.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_6.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_6.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_6.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_6.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_6.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_6.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_6.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_6.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_6.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_6.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_6.calculate('household_net_income', 2025)[0] - baseline_6.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 6: Single Parent with child age 3, $18k income

Federal EITC: $4,328.00
UT EITC (20% of federal): $865.60

Baseline (non-refundable):
  UT income tax before credits: $810.00
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $44,659.99

Reform (refundable for age <= 2):
  Has qualifying child: False
  UT refundable EITC: $0.00
  UT non-refundable EITC: $865.60
  UT non-refundable credits: $1,865.60
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $44,659.99

>>> BENEFIT FROM REFORM: $0.00


## Example 7: Childless Adult (Should NOT Benefit)

In [21]:
situation_7 = {
    "people": {
        "adult": {"age": {2025: 35}, "employment_income": {2025: 12_000}},
    },
    "tax_units": {"tax_unit": {"members": ["adult"]}},
    "households": {"household": {"members": ["adult"], "state_code": {2025: "UT"}}},
}

baseline_7 = Simulation(situation=situation_7)
reform_7 = Simulation(situation=situation_7, reform=create_reform())

print("="*70)
print("EXAMPLE 7: Childless Adult, $12k income")
print("="*70)
print(f"\nFederal EITC: ${baseline_7.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_7.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_7.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_7.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_7.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_7.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_7.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_7.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_7.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_7.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_7.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_7.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_7.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_7.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_7.calculate('household_net_income', 2025)[0] - baseline_7.calculate('household_net_income', 2025)[0]:,.2f}")

EXAMPLE 7: Childless Adult, $12k income

Federal EITC: $543.43
UT EITC (20% of federal): $108.69

Baseline (non-refundable):
  UT income tax before credits: $540.00
  UT non-refundable credits: $108.69
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $13,006.33

Reform (refundable for age <= 2):
  Has qualifying child: False
  UT refundable EITC: $0.00
  UT non-refundable EITC: $108.69
  UT non-refundable credits: $108.69
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $13,006.33

>>> BENEFIT FROM REFORM: $0.00


## Example 8: Qualifying Child but NO Income (Should NOT Benefit)

Even with a refundable EITC, households with no earned income should get $0 because the EITC phases in with income. This verifies we're only making the existing credit refundable, not creating a new credit.

In [22]:
situation_8 = {
    "people": {
        "parent": {"age": {2025: 28}, "employment_income": {2025: 0}},
        "child": {"age": {2025: 1}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_8 = Simulation(situation=situation_8)
reform_8 = Simulation(situation=situation_8, reform=create_reform())

print("="*70)
print("EXAMPLE 8: Single Parent with 1-year-old, $0 income")
print("="*70)
print(f"\nFederal EITC: ${baseline_8.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_8.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_8.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_8.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_8.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_8.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_8.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_8.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_8.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_8.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_8.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_8.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_8.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_8.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_8.calculate('household_net_income', 2025)[0] - baseline_8.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n*** Key check: Federal EITC = $0 because EITC phases in with earned income ***")
print(f"*** Therefore UT EITC = 20% * $0 = $0, and refundable EITC = $0 ***")

EXAMPLE 8: Single Parent with 1-year-old, $0 income

Federal EITC: $0.00
UT EITC (20% of federal): $0.00

Baseline (non-refundable):
  UT income tax before credits: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $28,042.52

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $0.00
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $28,042.52

>>> BENEFIT FROM REFORM: $0.00

*** Key check: Federal EITC = $0 because EITC phases in with earned income ***
*** Therefore UT EITC = 20% * $0 = $0, and refundable EITC = $0 ***


## Example 9: Qualifying Child with $5k Income (Should Benefit - Partial EITC)

With $5k income, the EITC is in the phase-in range. This household should benefit since they get a partial federal EITC that becomes refundable.

In [23]:
situation_9 = {
    "people": {
        "parent": {"age": {2025: 28}, "employment_income": {2025: 5_000}},
        "child": {"age": {2025: 1}, "is_tax_unit_dependent": {2025: True}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {2025: "UT"}}},
}

baseline_9 = Simulation(situation=situation_9)
reform_9 = Simulation(situation=situation_9, reform=create_reform())

print("="*70)
print("EXAMPLE 9: Single Parent with 1-year-old, $5k income (phase-in)")
print("="*70)
print(f"\nFederal EITC: ${baseline_9.calculate('eitc', 2025)[0]:,.2f}")
print(f"UT EITC (20% of federal): ${baseline_9.calculate('ut_eitc', 2025)[0]:,.2f}")
print(f"\nBaseline (non-refundable):")
print(f"  UT income tax before credits: ${baseline_9.calculate('ut_income_tax_before_credits', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${baseline_9.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${baseline_9.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${baseline_9.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${baseline_9.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\nReform (refundable for age <= 2):")
print(f"  Has qualifying child: {reform_9.calculate('ut_has_qualifying_child_for_refundable_eitc', 2025)[0]}")
print(f"  UT refundable EITC: ${reform_9.calculate('ut_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable EITC: ${reform_9.calculate('ut_non_refundable_eitc', 2025)[0]:,.2f}")
print(f"  UT non-refundable credits: ${reform_9.calculate('ut_non_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT refundable credits: ${reform_9.calculate('ut_refundable_credits', 2025)[0]:,.2f}")
print(f"  UT income tax: ${reform_9.calculate('ut_income_tax', 2025)[0]:,.2f}")
print(f"  Household net income: ${reform_9.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n>>> BENEFIT FROM REFORM: ${reform_9.calculate('household_net_income', 2025)[0] - baseline_9.calculate('household_net_income', 2025)[0]:,.2f}")
print(f"\n*** At $5k income, federal EITC is in phase-in range ***")
print(f"*** UT EITC = 20% of federal, and this becomes refundable ***")

EXAMPLE 9: Single Parent with 1-year-old, $5k income (phase-in)

Federal EITC: $1,700.00
UT EITC (20% of federal): $340.00

Baseline (non-refundable):
  UT income tax before credits: $225.00
  UT non-refundable credits: $1,340.00
  UT refundable credits: $0.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $0.00
  Household net income: $34,275.12

Reform (refundable for age <= 2):
  Has qualifying child: True
  UT refundable EITC: $340.00
  UT non-refundable EITC: $0.00
  UT non-refundable credits: $1,000.00
  UT refundable credits: $340.00


Invalid values for enum StateGroup: ['UT']. These will be encoded as index 0.


  UT income tax: $-340.00
  Household net income: $34,615.12

>>> BENEFIT FROM REFORM: $340.00

*** At $5k income, federal EITC is in phase-in range ***
*** UT EITC = 20% of federal, and this becomes refundable ***


## Summary Table

In [24]:
examples = [
    ("1: Child age 1, $20k", baseline_1, reform_1, True),
    ("2: Child age 5, $20k", baseline_2, reform_2, False),
    ("3: Infant age 0, $15k", baseline_3, reform_3, True),
    ("4: Kids age 1 & 7, $25k", baseline_4, reform_4, True),
    ("5: Child age 2, $18k", baseline_5, reform_5, True),
    ("6: Child age 3, $18k", baseline_6, reform_6, False),
    ("7: Childless, $12k", baseline_7, reform_7, False),
    ("8: Child age 1, $0", baseline_8, reform_8, False),
    ("9: Child age 1, $5k", baseline_9, reform_9, True),
]

summary_data = []
for desc, baseline, reform, expected_benefit in examples:
    baseline_income = baseline.calculate('household_net_income', 2025)[0]
    reform_income = reform.calculate('household_net_income', 2025)[0]
    benefit = reform_income - baseline_income
    federal_eitc = baseline.calculate('eitc', 2025)[0]
    ut_eitc = baseline.calculate('ut_eitc', 2025)[0]
    refundable_eitc = reform.calculate('ut_refundable_eitc', 2025)[0]
    actual_benefit = benefit > 0.5
    status = "PASS" if actual_benefit == expected_benefit else "FAIL"
    
    summary_data.append({
        "Example": desc,
        "Fed EITC": f"${federal_eitc:,.0f}",
        "UT EITC": f"${ut_eitc:,.0f}",
        "Refundable": f"${refundable_eitc:,.0f}",
        "Benefit": f"${benefit:,.2f}",
        "Status": status,
    })

df_summary = pd.DataFrame(summary_data)
print("="*90)
print("SUMMARY: UT Refundable EITC Household Examples")
print("="*90)
print(df_summary.to_string(index=False))
print("="*90)

all_pass = all(row["Status"] == "PASS" for row in summary_data)
print(f"\nAll tests passed: {all_pass}")

print("\n" + "-"*70)
print("KEY INSIGHT - EITC Phase-in Verification:")
print("-"*70)
print("Example 8 ($0 income):  Fed EITC = $0, Benefit = $0")
print("Example 9 ($5k income): Fed EITC phases in, Benefit > $0")
print("\nThis confirms the reform only makes the EXISTING credit refundable.")
print("The EITC phase-in structure is preserved.")
print("-"*70)

SUMMARY: UT Refundable EITC Household Examples
                Example Fed EITC UT EITC Refundable   Benefit Status
   1: Child age 1, $20k   $4,328    $866       $866   $865.60   PASS
   2: Child age 5, $20k   $4,328    $866         $0     $0.00   PASS
  3: Infant age 0, $15k   $4,328    $866       $866   $865.60   PASS
4: Kids age 1 & 7, $25k   $6,805  $1,361     $1,361 $1,360.91   PASS
   5: Child age 2, $18k   $4,328    $866       $866   $865.60   PASS
   6: Child age 3, $18k   $4,328    $866         $0     $0.00   PASS
     7: Childless, $12k     $543    $109         $0     $0.00   PASS
     8: Child age 1, $0       $0      $0         $0     $0.00   PASS
    9: Child age 1, $5k   $1,700    $340       $340   $340.00   PASS

All tests passed: True

----------------------------------------------------------------------
KEY INSIGHT - EITC Phase-in Verification:
----------------------------------------------------------------------
Example 8 ($0 income):  Fed EITC = $0, Benefit = $0
Ex

In [25]:
# Export summary table to CSV
df_summary.to_csv("ut_refundable_eitc_summary.csv", index=False)
print(f"Summary table exported to: ut_refundable_eitc_summary.csv")

Summary table exported to: ut_refundable_eitc_summary.csv
